# 4. Estimating Uncertainty In Classification Predictions (분류 예측의 불확실성 추정)
- 어떤 테스트 포인트에 대해 분류기가 예측한 클래스가 무엇인지 뿐만 아니라 정확한 클래스임을 얼마나 확신하는지가 중요할 때가 많음
- 실제 애플리케이션에서는 오류의 종류에 따라 전혀 다른 결과를 만듦
- 암을 진ㄷ단하는 의료 애플리케이션의 경우, 거짓 양성(False Positive) 예측은 환자에게 추가 진료를 요구하겠지만 거짓 음성(False Negative) 예측은 심각한 질병을 치료하지 못하게 만들 수 있음
- scikit-learn 분류기에서 불확실성을 추정할 수 있는 함수가 두 개 있음
- decision_function과 predict_proba임
- 대부분의(전체는 아니고) 분류 클래스는 적어도 둘 중 하나를 제공하고 두 함수를 모두 제공하는 경우도 많음

In [2]:
# 인위적으로 만든 2차원 데이터셋을 사용해 GradientBoostingClassifier 분류기의 decision_function과 predict_proba 메서드가 어떤 역할을 하는지 확인
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
import numpy as np
X, y = make_circles(noise=0.25, factor=0.5, random_state=1)

# 예제를 위해 클래스의 이름을 "blue"와 "red"로 바꿈
y_named = np.array(["blue", "red"])[y]

# 여러 개의 배열을 한꺼번에 train_test_split에 넣을 수 있음
# 훈련 세트와 테스트 세트로 나뉘는 방식은 모두 같음
X_train, X_test, y_train_named, y_test_named, y_train, y_test = train_test_split(X, y_named, y, random_state=0)

# 그레이디언트 부스팅 모델을 만듦
gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, y_train_named)

GradientBoostingClassifier(random_state=0)

## 4-1. 결정 함수
- 이진 분류에서 decision_function 반환값의 크기는 (n_sample,)이며 각 샘플이 하나의 실수 값을 반환

In [3]:
print("X_test.shape:", X_test.shape)
print("결정 함수 결과 형태:", gbrt.decision_function(X_test).shape)

X_test.shape: (25, 2)
결정 함수 결과 형태: (25,)


- 이 값은 모델이 데이터 포인트가 양성 클래스인 클래스 1에 속한다고 믿는 정도
- 양수 값은 양성 클래스를 의미하며 음수 값은 음성 (즉 다른) 클래스를 의미

In [4]:
# 결정 함수 결과 중 앞부분 일부를 확인
print("결정 함수:\n", gbrt.decision_function(X_test)[:6])

결정 함수:
 [ 4.13592603 -1.67785652 -3.95106099 -3.62604651  4.28986642  3.66166081]


In [5]:
# 결정 함수의 부호만 보고 예측 결과를 알 수 있음
print("임계치와 결정 함수 결과 비교:\n", gbrt.decision_function(X_test) > 0)
print("예측:\n", gbrt.predict(X_test))

임계치와 결정 함수 결과 비교:
 [ True False False False  True  True False  True  True  True False  True
  True False  True False False False  True  True  True  True  True False
 False]
예측:
 ['red' 'blue' 'blue' 'blue' 'red' 'red' 'blue' 'red' 'red' 'red' 'blue'
 'red' 'red' 'blue' 'red' 'blue' 'blue' 'blue' 'red' 'red' 'red' 'red'
 'red' 'blue' 'blue']


In [6]:
# 이진 분류에서 음성 클래스는 항상 classes_ 속성의 첫 번째 원소이고 양성 클래스는 classes_의 두 번째 원소
# 그래서 predict 함수의 결과를 완전히 재현하려면 classes_ 속성을 사용
# boolean 값을 0과 1로 반환
greater_zero = (gbrt.decision_function(X_test) > 0).astype(int)
# classes_에 인덱스로 사용
pred = gbrt.classes_[greater_zero]
# pred와 gbrt.predict의 결과를 비교
print("pred는 예측 결과와 같다:", np.all(pred == gbrt.predict(X_test)))

pred는 예측 결과와 같다: True
